In [ ]:
import os
import json
import re
import pandas as pd

In [ ]:
def load_solomon_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if "VEHICLE" in line:
            vehicle_line = lines[i + 2]
            break

    vehicle_number_line = vehicle_line.split()
    vehicle_number = int(vehicle_number_line[0])
    vehicle_capacity = int(vehicle_number_line[1])
    vehicle_capacitiesLB = [vehicle_capacity] * vehicle_number

    demandLB = []
    time_windows = []
    coords = []
    due_time_0 = 0

    for line in lines[i + 4:]:
        parts = line.split()
        if len(parts) == 7:
            customer_id = int(parts[0])
            x_coord = int(parts[1])
            y_coord = int(parts[2])
            demand = int(parts[3])
            ready_time = int(parts[4])
            due_time = int(parts[5])
            service_time = int(parts[6])

            if customer_id == 0:
                due_time_0 = due_time
            demandLB.append(demand)
            time_windows.append((ready_time, due_time))
            coords.append((x_coord, y_coord))

    return vehicle_number, vehicle_capacity
    # return vehicle_number, vehicle_capacitiesLB, demandLB, time_windows, coords, service_time, due_time_0

In [ ]:
def process_text(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    processed_lines = []
    for line in lines:
        # 保留换行符后，去掉每行中的多余空格，将多个空格替换为一个空格
        # After retaining the newline character, remove the extra Spaces in each line and replace multiple Spaces with one space
        line = re.sub(r'\s+', ' ', line.strip())  # After removing the first and last Spaces, replace the multiple Spaces in the middle
        
        # Restore the line break character
        line = line + '\n'

        if "READY TIME DUE DATE SERVICE TIME" in line:
            line = line.replace('READY TIME DUE DATE SERVICE TIME', 'READY_TIME DUE_DATE SERVICE_TIME')

        # Add the processed rows to the list
        processed_lines.append(line)

    processed_text = ''.join(processed_lines)

    # Print out the complete text
    return processed_text

In [ ]:
import os
import json
import matplotlib.pyplot as plt
from collections import Counter

# Path setting
save_path = "/home/wsl/CVRPTW/data/routes_distance_200"

# Store the last run_time of each file
last_run_times = []

# Confirm whether save_path exists and is a folder
if not os.path.exists(save_path) or not os.path.isdir(save_path):
    print(f"Error: {save_path} is not a valid directory.")
else:
    # Traverse the files in the "result" folder
    for file_name in os.listdir(save_path):
        result_file_path = os.path.join(save_path, file_name)

        # Only handle files ending with _results.txt
        if file_name.endswith("_results.txt"):
            try:
                with open(result_file_path, 'r') as result_file:
                    lines = [line.strip() for line in result_file.readlines() if line.strip()]

                # Make sure there are at least three lines of data
                if len(lines) >= 3:
                    # Obtain the last group of three lines of data
                    run_time_line = lines[-3]

                    # Extract the value of run_time
                    run_time = int(run_time_line.split(":")[1].strip())
                    last_run_times.append(run_time)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

# Count the occurrence times of run_time
run_time_counts = Counter(last_run_times)

# Print the result
for run_time, count in run_time_counts.items():
    print(f"run_time {run_time} appears {count} times")

# Draw the histogram
plt.figure(figsize=(10, 6))
plt.hist(last_run_times, bins=range(min(last_run_times), max(last_run_times) + 1), color='blue', alpha=0.7, edgecolor='black')
plt.xlabel("Run Time")
plt.ylabel("Frequency")
plt.title("Ortools high_quality solution run time of Solomon100")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import os
import json
import pandas as pd

# Read the result_distance.xlsx file
result_distance_df = pd.read_excel("/home/wsl/CVRPTW/result_distance.xlsx")
result_distance_dict = dict(zip(result_distance_df['file_name'], result_distance_df['distance']))

result_split_path = "/home/wsl/CVRPTW/data/result_single_0406"
result_path = "/home/wsl/CVRPTW/data/result_single_0406"
origine_data_path = "/home/wsl/CVRPTW/data/origine data/solomon_100"
best_solution_route_path = "/home/wsl/CVRPTW/data/best_solution_route"

customer_number = 100

all_conversations = []

for file_name in os.listdir(best_solution_route_path):
    if file_name.endswith(".txt"):
        print(f"Processing file: {file_name}")

        file_name_without_extension = file_name.replace('.txt', '')
        file_link = os.path.join(origine_data_path, f"{file_name_without_extension}.txt")

        file_content = process_text(file_link)

        vehicle_number, vehicle_capacity = load_solomon_data(file_link)

        total_distance = result_distance_dict.get(file_name_without_extension, None)

        try:
            with open(os.path.join(best_solution_route_path, file_name), 'r') as best_solution_file:
                all_routes_versions = [line.strip() for line in best_solution_file.readlines() if line.strip()]
                all_routes_list = [json.loads(route) for route in all_routes_versions]

            file_conversations = []

            for all_routes in all_routes_list:
                gpt_value = json.dumps(all_routes)
                human_value = f"""Data: {file_content}; \n Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.\n The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated.
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list."""  # 可以根据需要自定义 human_value

                conversation = {
                    "conversations": [
                        {
                            "from": "human",
                            "value": human_value
                        },
                        {
                            "from": "gpt",
                            "value": gpt_value
                        }
                    ],
                    "system": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."  # 系统消息内容
                }

                file_conversations.append(conversation)

            all_conversations.extend(file_conversations)

            output_json_path_file = os.path.join(result_split_path, f"conversations_output_{file_name}.json")
            with open(output_json_path_file, 'w') as json_file:
                json.dump(file_conversations, json_file, indent=4)

            print(f"Conversation for {file_name} has been saved to '{output_json_path_file}'")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

output_json_path = os.path.join(result_path, "conversations_output_single_100_0404.json")
with open(output_json_path, 'w') as json_file:
    json.dump(all_conversations, json_file, indent=4)
print(f"All conversations have been saved to '{output_json_path}'")


In [ ]:
save_path = "/home/wsl/CVRPTW/data/result_route_0402/c"
result_split_path = "/home/wsl/CVRPTW/data/result_single_0404_1"
result_path = "/home/wsl/CVRPTW/data/merge_result_single_0404_1"
origine_data_path = "/home/wsl/CVRPTW/data/origine data/solomon_100"

customer_number = 100

all_conversations = []

if not os.path.exists(save_path) or not os.path.isdir(save_path):
    print(f"Error: {save_path} is not a valid directory.")
else:
    for file_name in os.listdir(save_path):
        result_file_path = os.path.join(save_path, file_name)

        if file_name.endswith("_results.txt"):
            print(f"Processing file: {result_file_path}")

            base_name = os.path.basename(result_file_path)

            file_name_without_results = base_name.replace('_results.txt', '')

            file_link = os.path.join(origine_data_path, file_name_without_results)
            file_content = process_text(file_link)

            vehicle_number, vehicle_capacity = load_solomon_data(file_link)


            try:
                with open(result_file_path, 'r') as result_file:
                    lines = [line for line in result_file.readlines() if line.strip()]

                file_conversations = []

                for i in range(0, len(lines), 3):
                    if i + 2 < len(lines):  
                        run_time_line = lines[i].strip()
                        all_routes_line = lines[i + 1].strip()
                        total_distance_line = lines[i + 2].strip()

                        try:
                            run_time = int(run_time_line.split(":")[1].strip())
                            all_routes = json.loads(all_routes_line.split(":")[1].strip())
                            total_distance = float(total_distance_line.split(":")[1].strip())

                            gpt_value = json.dumps(all_routes)
                            human_value = f"""Data: {file_content}; \n Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.\n The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated.
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list."""

                            conversation = {
                                "conversations": [
                                    {
                                        "from": "human",
                                        "value": human_value
                                    },
                                    {
                                        "from": "gpt",
                                        "value": gpt_value
                                    }
                                ],
                                "system": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."  # 系统消息内容，可以根据需要调整
                            }

                            file_conversations.append(conversation)

                        except Exception as e:
                            print(f"Error parsing run_time or total_distance in {result_file_path}: {e}")

                all_conversations.extend(file_conversations)

                output_json_path_file = os.path.join(result_split_path, f"conversations_output_{file_name}.json")
                with open(output_json_path_file, 'w') as json_file:
                    json.dump(file_conversations, json_file, indent=4)

                print(f"Conversation for {file_name} has been saved to '{output_json_path_file}'")

            except Exception as e:
                print(f"Error processing {result_file_path}: {e}")

    output_json_path = os.path.join(result_path, "conversations_output_single_100_0404.json")
    with open(output_json_path, 'w') as json_file:
        json.dump(all_conversations, json_file, indent=4)

In [ ]:
save_path = "/home/wsl/CVRPTW/data/result_route_processed_0404"
result_split_path = "/home/wsl/CVRPTW/data/result_multi_0404"
result_path = "/home/wsl/CVRPTW/data/merge_result_multi_0404"
origine_data_path = "/home/wsl/CVRPTW/data/origine data/solomon_100"
       
customer_number = 100

all_conversations = []

if not os.path.exists(save_path) or not os.path.isdir(save_path):
    print(f"Error: {save_path} is not a valid directory.")
else:
    for file_name in os.listdir(save_path):
        result_file_path = os.path.join(save_path, file_name)

        if file_name.endswith("_results.txt"):
            print(f"Processing file: {result_file_path}")

            base_name = os.path.basename(result_file_path)

            file_name_without_results = base_name.replace('_results.txt', '')

            file_link = os.path.join(origine_data_path, file_name_without_results)
            file_content = process_text(file_link)

            vehicle_number, vehicle_capacity = load_solomon_data(file_link)


            try:
                with open(result_file_path, 'r') as result_file:
                    lines = [line for line in result_file.readlines() if line.strip()]

                conversation = {
                    "conversations": [],
                    "system": (
                        "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."
                    )
                }

                all_distance = []
                j = 0
                first_run_time = None 
                for i in range(0, len(lines), 3):
                    if i + 2 < len(lines):  
                        run_time_line = lines[i].strip()
                        all_routes_line = lines[i + 1].strip()
                        total_distance_line = lines[i + 2].strip()

                        try:
                            run_time = int(run_time_line.split(":")[1].strip())
                            all_routes = json.loads(all_routes_line.split(":")[1].strip())
                            total_distance = float(total_distance_line.split(":")[1].strip())
                            all_distance.append(total_distance)
                            print(all_distance)

                            gpt_value = json.dumps(all_routes)

                            if first_run_time is None:
                                first_run_time = run_time

                            if run_time == first_run_time:
                                human_value = f"""Data: {file_content}; \n Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.\n The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated.
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. """
                            else:
                                human_value = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[j-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

                            conversation["conversations"].append({
                                "from": "human",
                                "value": human_value
                            })

                            
                            conversation["conversations"].append({
                                "from": "gpt",
                                "value": gpt_value 
                            })


                        except Exception as e:
                            print(f"Error parsing run_time or total_distance in {result_file_path}: {e}")
                        
                        j += 1

                conversation["conversations"].append({
                    "from": "human",
                    "value": f"Excellent! In the last round of dialogue, your output helped me get a high-quality solution for my algorithm, where the total_distance (best cost) that I got was {all_distance[j-1]}."
                })

                conversation["conversations"].append({
                    "from": "gpt",
                    "value": "Well, I will continue to consider the internal relationships of the data you provided and calculate a more high-quality solution."
                })

                all_conversations.append(conversation)

                output_json_path_file = os.path.join(result_split_path, f"conversations_output_multi_reset_{file_name}.json")
                with open(output_json_path_file, 'w') as json_file:
                    json.dump(conversation, json_file, indent=4)

                print(f"Conversation for {file_name} has been saved to '{output_json_path_file}'")

            except Exception as e:
                print(f"Error processing {result_file_path}: {e}")
                
    output_json_path = os.path.join(result_path, "conversations_output_multi_100_0404.json")
    with open(output_json_path, 'w') as json_file:
        json.dump(all_conversations, json_file, indent=4)

    print(f"All conversations have been saved to '{output_json_path}'")

In [ ]:
import os
import json
import random

# 文件夹路径
folder_path = "/home/wsl/CVRPTW/data/best111"

# 遍历该路径中的所有文件
for file_name in os.listdir(folder_path):
    # 确保只处理 .txt 文件
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_path}")
        
        try:
            # 读取文件内容，假设文件内容是一个列表形式的 Route 数据
            with open(file_path, 'r') as instance_file:
                route_lists = json.load(instance_file)  # 从文件中读取数据

            # 打乱生成不同版本并存储
            all_permutations = set()
            # 将原始的 route_lists 转为元组，以确保它是不可变的，可以存入集合中
            all_permutations.add(tuple(tuple(route) for route in route_lists))  # 将每个 route 转为元组

            # 打开文件进行写入，覆盖原文件内容
            with open(file_path, 'w') as instance_file:
                # 先存储原始顺序
                json.dump(route_lists, instance_file)
                instance_file.write('\n\n')  # 用换行隔开

                # 打乱并生成多个版本
                for _ in range(30):  # 限制最大打乱次数，避免无限打乱
                    shuffled_routes = route_lists[:]  # 拷贝一份原始数据
                    random.shuffle(shuffled_routes)  # 打乱
                    # 将打乱后的版本转为不可变元组并确保唯一
                    shuffled_tuple = tuple(tuple(route) for route in shuffled_routes)
                    if shuffled_tuple not in all_permutations:
                        all_permutations.add(shuffled_tuple)
                        instance_file.write(json.dumps(shuffled_routes) + '\n\n')  # 存储打乱版本

            print(f"Processed file saved to: {file_path}")

        except Exception as e:
            print(f"Error processing {file_path}: {e}")


In [ ]:
import os
import json
import pandas as pd

result_distance_df = pd.read_excel("/home/wsl/CVRPTW/result_distance.xlsx")
result_distance_dict = dict(zip(result_distance_df['file_name'], result_distance_df['distance']))

source_path = "/home/wsl/CVRPTW/data/result_route_0402/c" 
destination_path = "/home/wsl/CVRPTW/data/result_route_processed" 
best_solution_route_path = "/home/wsl/CVRPTW/data/best_solution_route"  

os.makedirs(destination_path, exist_ok=True)

for file_name in os.listdir(source_path):
    if file_name.endswith("_results.txt"):
        print(f"Processing file: {file_name}")

        result_file_path = os.path.join(source_path, file_name)
        
        file_name_without_extension = file_name.replace('.txt_results.txt', '').lower()

        best_solution_file_path = os.path.join(best_solution_route_path, f"{file_name_without_extension}.txt")
        
        with open(best_solution_file_path, 'r') as best_solution_file:
            best_solution_lines = best_solution_file.readlines()
            all_routes_first_version = json.loads(best_solution_lines[0].strip())
        
        total_distance = result_distance_dict.get(file_name_without_extension, None)

        try:
            with open(result_file_path, 'r') as result_file:
                lines = [line.strip() for line in result_file.readlines() if line.strip()]

            result_data = []
            for i in range(0, len(lines), 3):
                if i + 2 < len(lines): 
                    run_time_line = lines[i]
                    all_routes_line = lines[i + 1]
                    total_distance_line = lines[i + 2]

                    run_time = int(run_time_line.split(":")[1].strip())
                    all_routes = json.loads(all_routes_line.split(":")[1].strip())
                    total_distance_value = float(total_distance_line.split(":")[1].strip())

                    result_data.append([run_time, all_routes, total_distance_value])

            result_data.append([0, all_routes_first_version, total_distance])

            output_file_path = os.path.join(destination_path, file_name)

            with open(output_file_path, 'w') as output_file:
                for run_time, all_routes, total_distance_value in result_data:
                    output_file.write(f"Run Time: {run_time}\n")
                    output_file.write(f"All Routes: {json.dumps(all_routes)}\n")
                    output_file.write(f"Total Distance: {total_distance_value}\n\n")

            print(f"Processed file saved to: {output_file_path}")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

file_path = '/home/wsl/CVRPTW/data/test_data_100/c109.txt' 
with open(file_path, 'r') as file:
    data0 = file.read()

data = []
for line in data0.strip().split('\n'):
    data.append(line.split())

columns = ["CUST", "XCOORD.", 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df = pd.DataFrame(data[9:], columns=columns)

numeric_cols = ['CUST', 'XCOORD.', 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df['sum'] = df['DUE'] - df['READY']
print(np.mean(df['sum']))

plt.title(f'Time window distribution of C109')
plt.barh(df.index, df['DUE'] - df['READY'], left=df['READY'], color='blue', alpha=0.7)
plt.show()

plt.scatter(df.loc[1:, 'XCOORD.'], df.loc[1:, 'YCOORD.'], label='Customers', marker='o', color='blue')
plt.scatter(df.loc[0, 'XCOORD.'], df.loc[0, 'YCOORD.'], label='Depot', marker='x', color='red')
for i, row in df.iterrows():
    plt.annotate(row['CUST'], (row['XCOORD.'], row['YCOORD.']), textcoords="offset points", xytext=(0, 5), ha='center')
plt.title(f'Customer and Depot Locations of C109')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

file_name = "r211"
file_path = f'/home/wsl/CVRPTW/data/test_data_100/{file_name}.txt'
with open(file_path, 'r') as file:
    data0 = file.read()

data = []
for line in data0.strip().split('\n'):
    data.append(line.split())

columns = ["CUST", "XCOORD.", 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df = pd.DataFrame(data[9:], columns=columns)

numeric_cols = ['CUST', 'XCOORD.', 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df['sum'] = df['DUE'] - df['READY']
print(np.mean(df['sum']))

plt.title(f'Time window distribution of {file_name}')
plt.barh(df.index, df['DUE'] - df['READY'], left=df['READY'], color='blue', alpha=0.7)
plt.show()

plt.scatter(df.loc[1:, 'XCOORD.'], df.loc[1:, 'YCOORD.'], label='Customers', marker='o', color='blue')
plt.scatter(df.loc[0, 'XCOORD.'], df.loc[0, 'YCOORD.'], label='Depot', marker='x', color='red')
for i, row in df.iterrows():
    plt.annotate(row['CUST'], (row['XCOORD.'], row['YCOORD.']), textcoords="offset points", xytext=(0, 5), ha='center')
plt.title(f'Customer and Depot Locations of {file_name}')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

file_name = "rc203"
file_path = f'/home/wsl/CVRPTW/data/test_data_100/{file_name}.txt' 
with open(file_path, 'r') as file:
    data0 = file.read()

data = []
for line in data0.strip().split('\n'):
    data.append(line.split())

columns = ["CUST", "XCOORD.", 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df = pd.DataFrame(data[9:], columns=columns)

numeric_cols = ['CUST', 'XCOORD.', 'YCOORD.', 'DEMAND', 'READY', 'DUE', 'SERVICE']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df['sum'] = df['DUE'] - df['READY']
print(np.mean(df['sum']))

plt.title(f'Time window distribution of {file_name}')
plt.barh(df.index, df['DUE'] - df['READY'], left=df['READY'], color='blue', alpha=0.7)
plt.show()

plt.scatter(df.loc[1:, 'XCOORD.'], df.loc[1:, 'YCOORD.'], label='Customers', marker='o', color='blue')
plt.scatter(df.loc[0, 'XCOORD.'], df.loc[0, 'YCOORD.'], label='Depot', marker='x', color='red')
for i, row in df.iterrows():
    plt.annotate(row['CUST'], (row['XCOORD.'], row['YCOORD.']), textcoords="offset points", xytext=(0, 5), ha='center')
plt.title(f'Customer and Depot Locations of {file_name}')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.legend()
plt.show()